# Pickhardt Payments Example

Example code demonstrating how to use the `pickhardtpayments` package in python. You need to install the library via `pip install pickhardtpayments` or you can download the full source code at https://ln.rene-pickhardt.de or a copy from github at: https://www.github.com/renepickhardt/pickhardtpayments

Of course you can use the classes int the library to create your own async payment loop or you could exchange the Oracle to talk to the actual Lightning network by wrapping against your favourite node implementation and exposing the the `send_onion` call. 

This example assumes a randomly generated Oracle to conduct payments in a simulated way. For this you will need an actual channelgraph which you can get for example with `lightning-cli listchannels > listchannels20220412.json`

## Acknowledgements & Funding
This work is funded via various sources including NTNU & BitMEX as well as many generous donors via https://donate.ln.rene-pickhardt.de or https://www.patreon.com/renepickhardt Feel free to go to my website at https://ln.rene-pickhardt.de to learn how I have been contributing to the open source community and why it is important to have independent open source contributors. In case you also wish to support me I will be very grateful

In [2]:
sys.path.append('..')
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession

In [3]:
#Rene Pickhardt's public node key
RENE = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#Carsten Otto's public node key
C_OTTO = "027ce055380348d7812d2ae7745701c9f93e70c1adeb2657f053f91df4f2843c71"

#we first need to import the chanenl graph from c-lightning jsondump
#you can get your own data set via:
# $: lightning-cli listchannels > listchannels20220412.json
# alternatively you can go to https://ln.rene-pickhardt.de to find a data dump
channel_graph = ChannelGraph("listchannels20220412.json")

#we now create ourself an Oracle. This is Simulated Network that assumes unformly distribution 
#of the liquidity for the channels on the `channel_graph`. Of course one could create ones own 
#oracle (for example a wrapper to an existing lightning network node / implementation)
oracle_lightning_network = OracleLightningNetwork(channel_graph)

In [4]:
# Since we randomly generated our Oracle but also since we control it we can compute
# The maximum possible amout that can be payed between two nodes
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,1000)
print(maximum_payable_amount, "sats would be possible on this oracle to deliver if including 1 sat basefee channels")


130218715 sats would be possible on this oracle to deliver if including 1 sat basefee channels


In [5]:
#Of course we want to restrict ourselves to the zeroBaseFee part of the network.
#Therefor we compute the theoretical maximum payable amount for that subgraph
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,0)
print(maximum_payable_amount, "sats possible on this oracle on the zeroBaseFeeGraph")

79191788 sats possible on this oracle on the zeroBaseFeeGraph


In [6]:
# We chose an amount that is 50% of half the theoretic maximum to demonstrate the the
# minimum cost flow solver with Bayesian updates on the Uncertainty Network finds the
# liquidity rather quickly
tested_amount = int(maximum_payable_amount/2)

# From the channel graph we can derrive our initial Uncertainty Network which is the main data structure
# that we maintain in order to deliver sats from one node to another
uncertainty_network = UncertaintyNetwork(channel_graph)

#we create ourselves a payment session which in this case operates by sending out the onions
#sequentially 
payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False)

#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=0,base=0)

Round number:  1
Try to deliver 39595894 satoshi:

Statistics about 12 candidate onions:

successful attempts:
--------------------
 p =  52.92% amt:   2800000 sats  hops: 3 ppm:  1897
 p =  68.30% amt:   2020000 sats  hops: 3 ppm:  1974
 p =  69.25% amt:    630528 sats  hops: 4 ppm:  3717
 p =  59.67% amt:    200000 sats  hops: 4 ppm: 21787
 p =  70.31% amt:   1000000 sats  hops: 4 ppm:  2060
 p =  66.80% amt:   1000000 sats  hops: 4 ppm:  2187
 p =  48.21% amt:   3355443 sats  hops: 4 ppm:  2496
 p =  44.73% amt:   3000000 sats  hops: 5 ppm:  2580
 p =  20.97% amt:   5046329 sats  hops: 5 ppm:  2551
 p =  43.76% amt:    969011 sats  hops: 5 ppm:  2057
 p =  16.96% amt:   5369472 sats  hops: 6 ppm:  5317

failed attempts:
----------------
 p =   7.86% amt:  14205111 sats  hops: 3 ppm:  2036 

Attempt Summary:

Tried to deliver   39595894 sats
expected to deliver   11257371 sats 	(28.43%)
actually deliverd   25390783 sats 	(64.12%)
deviation: 2.26
planned_fee: 108729.432 sat
paid fees:

## Optimizing for Fees

controlling mu we can decide how much we wish to focuse on lower fees. However we will see that it will be much harder to deliver the same amount in the sense that we need to send out more onions and also have more failed attampts. Consiquantly we expect to need more time.

In [7]:
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0)

Round number:  1
Try to deliver 39595894 satoshi:

Statistics about 31 candidate onions:

successful attempts:
--------------------
 p =  78.46% amt:    192000 sats  hops: 2 ppm:  1342
 p =  72.00% amt:    200000 sats  hops: 2 ppm:  1357
 p =  73.92% amt:    194200 sats  hops: 3 ppm:  1362
 p =  58.88% amt:    400000 sats  hops: 3 ppm:  1381
 p =  57.60% amt:   1000000 sats  hops: 3 ppm:  1507
 p =  47.15% amt:    900000 sats  hops: 3 ppm:  1416
 p =  18.95% amt:    740000 sats  hops: 3 ppm:  1381
 p =  68.13% amt:    152521 sats  hops: 4 ppm:  1410
 p =  71.16% amt:      7215 sats  hops: 4 ppm:  1417
 p =  37.09% amt:   2000000 sats  hops: 4 ppm:  1376
 p =  51.79% amt:    198808 sats  hops: 4 ppm:  1356
 p =  21.38% amt:    100264 sats  hops: 4 ppm:  1385
 p =  10.66% amt:   1000000 sats  hops: 4 ppm:  1368
 p =  23.26% amt:    568823 sats  hops: 4 ppm:  1340
 p =   7.37% amt:   2060306 sats  hops: 5 ppm:  1352
 p =   1.25% amt:   1510000 sats  hops: 6 ppm:  1352
 p =   8.94% amt:   

In [ ]:

payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False,
                                 use_negative_circle_solver=True)
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0,)